In [2]:
import pandas as pd

venues_data = pd.read_csv('C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/csv_files/venues_details.csv')
venues_data

,complex_id,venue_id,venue_name,venue_city_name,venue_country_name,venue_country_code,venue_timezone
0,sr:complex:705,sr:venue:70045,Cancha Central,Santiago,Chile,CHL,America/Santiago
1,sr:complex:1078,sr:venue:74858,Court One,Seville,Spain,ESP,Europe/Madrid
2,sr:complex:1495,sr:venue:62153,TC Dynamo,Saint Petersburg,Russia,RUS,Europe/Moscow
3,sr:complex:2375,sr:venue:77437,Campo 8,Oeiras,Portugal,PRT,Europe/Lisbon
4,sr:complex:4032,sr:venue:77437,Campo 8,Oeiras,Portugal,PRT,Europe/Lisbon
...,...,...,...,...,...,...,...
652,sr:complex:82356,sr:venue:82392,Loughborough University - Court 05,Loughborough,England,ENG,Europe/London
653,sr:complex:82415,sr:venue:82451,Mitchell Tennis Center - Texas AM - Court 04,"College Station, TX",USA,USA,America/Chicago
654,sr:complex:82417,sr:venue:82479,National Tennis Center - Outdoor - Court 06,Bucharest,Romania,ROU,Europe/Bucharest
655,sr:complex:82467,sr:venue:82495,Court 2,Saint Malo,France,FRA,Europe/Paris


save data in postgresql database

In [2]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


Creating DataBase:

In [3]:
pip install toml

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import psycopg2
import streamlit as st
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

import json
import toml

import sys
import os

# Load environment
try:
    with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/config.json", "r") as f:
        config = json.load(f)

    env = config["environment"] # 'local' or 'cloud'
except FileNotFoundError:
    raise FileNotFoundError("⚠️ 'config.json' not found. Please check the path.")
except json.JSONDecodeError:
    raise ValueError("⚠️ 'config.json' is not a valid JSON file.")
except KeyError:
    raise KeyError("⚠️ 'environment' key not found in config.json.")

# Load secrets from your custom toml file
local_secrets = toml.load("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/.pwd/pwd.toml")



# Append the directory where db_utils.py is located
sys.path.append(os.path.abspath('C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar'))


from db_utils import connect_to_postgres, connect_to_sportradar

# Connect to default DB and create target DB if needed
my_db_connection = connect_to_postgres(env, local_secrets) #connect to existing database first

# Set autocommit so CREATE DATABASE works
my_db_connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Create cursor and check for database existence
mycursor = my_db_connection.cursor()
mycursor.execute("SELECT * FROM pg_database WHERE datname='sportradar'")
exists = mycursor.fetchone()

if not exists:
    mycursor.execute("CREATE DATABASE sportradar")   #create target DB
    print("Database 'sportradar' created")
else:
    print("Database 'sportradar' already exists")

mycursor.close()
my_db_connection.close()




# Reconnect to the newly ensured 'sportradar' database
my_db_connection = connect_to_sportradar(env, local_secrets)

my_db_connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

#I need to connect the db and the python:
mycursor = my_db_connection.cursor()

print("Connected to 'sportradar' database.")

Database 'sportradar' already exists
Connected to 'sportradar' database.


Creating Venues Table:

In [5]:
venues_create_table_sql = """create table if not exists venues_table(
complex_id varchar(50) , 
venue_id varchar(50) PRIMARY KEY,
venue_name varchar(100) NOT NULL, 
venue_city_name varchar(100) NOT NULL, 
venue_country_name varchar(100) NOT NULL,
venue_country_code char(3),
venue_timezone varchar(100) NOT NULL,
FOREIGN KEY(complex_id) references complexes_table(complex_id))"""

#execute:
mycursor.execute(venues_create_table_sql)

print("Venues Table has been created")

Venues Table has been created


Fetching Column Name from venues_table:

In [6]:
column_name = """SELECT column_name FROM 
        information_schema.columns WHERE table_name = 'venues_table';"""
mycursor.execute(column_name)
fetching_column_names = mycursor.fetchall()
fetching_column_names

[('complex_id',),
 ('venue_id',),
 ('venue_name',),
 ('venue_city_name',),
 ('venue_country_name',),
 ('venue_country_code',),
 ('venue_timezone',)]

insert values into venues table:

In [7]:
venue_insert_table_sql = """Insert into venues_table(
complex_id, 
venue_id,
venue_name,
venue_city_name,
venue_country_name,
venue_country_code,
venue_timezone) 
values (%s, %s, %s, %s, %s, %s, %s) 
ON CONFLICT (venue_id) DO UPDATE SET
    complex_id = EXCLUDED.complex_id,
    venue_name = EXCLUDED.venue_name,
    venue_city_name = EXCLUDED.venue_city_name,
    venue_country_name = EXCLUDED.venue_country_name,
    venue_country_code = EXCLUDED.venue_country_code,
    venue_timezone = EXCLUDED.venue_timezone;"""


#ON CONFLICT (venue_id) DO UPDATE SET
#If the row I'm trying to insert would violate a uniqueness constraint (like a duplicate primary key), then update the existing row, instead of throwing an error.


#venue_name = EXCLUDED.venue_name:-
# …it means: "update venue_name with the value I tried to insert."

In [8]:
for index, row in venues_data.iterrows():
    print(row) #We will get the o/p in the format of dictionary with column names as keys.

    print("Inserting:", tuple(row))  #The iter row (only values) will converts into python tuple 
    print("----------------------\n")
    mycursor.execute(venue_insert_table_sql, tuple(row)) #('sr:competition:620', 'Hopman Cup', nan, 'mixed', 'mixed', 'sr:category:181') this tuple of rows will be given to insert query values.
print("Data inserted  into the venues_table using iterrows()")


#.iterrows() allows you to loop through each row of the table.
     # index: The index of the row (e.g., 0, 1, 2...)
     # row: A Pandas Series containing the data for that row (column names are keys).

#tuple(row) turns that row into a Python tuple — needed by SQL

complex_id              sr:complex:705
venue_id                sr:venue:70045
venue_name              Cancha Central
venue_city_name               Santiago
venue_country_name               Chile
venue_country_code                 CHL
venue_timezone        America/Santiago
Name: 0, dtype: object
Inserting: ('sr:complex:705', 'sr:venue:70045', 'Cancha Central', 'Santiago', 'Chile', 'CHL', 'America/Santiago')
----------------------

complex_id            sr:complex:1078
venue_id               sr:venue:74858
venue_name                  Court One
venue_city_name               Seville
venue_country_name              Spain
venue_country_code                ESP
venue_timezone          Europe/Madrid
Name: 1, dtype: object
Inserting: ('sr:complex:1078', 'sr:venue:74858', 'Court One', 'Seville', 'Spain', 'ESP', 'Europe/Madrid')
----------------------

complex_id             sr:complex:1495
venue_id                sr:venue:62153
venue_name                   TC Dynamo
venue_city_name       Saint Pe

Fetching Data from venues_table:

In [9]:
#execute:
mycursor.execute("Select * from venues_table")

fetching_Data = mycursor.fetchall()
fetching_Data

[('sr:complex:705',
  'sr:venue:70045',
  'Cancha Central',
  'Santiago',
  'Chile',
  'CHL',
  'America/Santiago'),
 ('sr:complex:1078',
  'sr:venue:74858',
  'Court One',
  'Seville',
  'Spain',
  'ESP',
  'Europe/Madrid'),
 ('sr:complex:1495',
  'sr:venue:62153',
  'TC Dynamo',
  'Saint Petersburg',
  'Russia',
  'RUS',
  'Europe/Moscow'),
 ('sr:complex:5520',
  'sr:venue:77437',
  'Campo 8',
  'Oeiras',
  'Portugal',
  'PRT',
  'Europe/Lisbon'),
 ('sr:complex:5526',
  'sr:venue:55223',
  'Court 2',
  'Ostrava',
  'Czechia',
  'CZE',
  'Europe/Prague'),
 ('sr:complex:6310',
  'sr:venue:77285',
  'Table 8',
  'Varazdin',
  'Croatia',
  'HRV',
  'Europe/Zagreb'),
 ('sr:complex:6888',
  'sr:venue:39934',
  'Court 3',
  'Montpellier',
  'France',
  'FRA',
  'Europe/Paris'),
 ('sr:complex:6904',
  'sr:venue:74197',
  'Court 1',
  'Orleans',
  'France',
  'FRA',
  'Europe/Paris'),
 ('sr:complex:7778',
  'sr:venue:72571',
  'Pista 4',
  'Malaga',
  'Spain',
  'ESP',
  'Europe/Madrid'),
 ('

close cursor & db connection:

In [10]:
mycursor.close()
my_db_connection.close()